In [71]:
from bs4 import BeautifulSoup
import requests
import re

In [82]:
url_dave = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-of-1930-1939.html'

headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get(url_dave, headers=headers)

response.status_code

200

In [83]:
soup = BeautifulSoup(response.content)

In [198]:
#Obtengo un set de los tags presentes en el tag 'p'
tags = soup.find_all(name="p")

#Separa en elementos de una lista las canciones.
songs_30s = tags[5].text.split('\n')

#Elimina los espacios de los extremos de cada elemento de la lista si
#los hubiese y devuelve una lista nueva con cada elemento
list_songs_30s = [i.strip() for i in songs_30s]

#Elimina los elementos vacios de la lista
list_songs_30s = list(filter(None, list_songs_30s))
#songs_30s = songs.split('\n')

list_songs_30s

['1. Over the Rainbow...Judy Garland (1939)',
 '2. Night and Day...Fred Astaire with Leo Reisman (1932)',
 '3. In the Mood...Glenn Miller (1939)',
 '4. Cheek to Cheek...Fred Astaire with Leo Reisman (1935)',
 "5. Stormy Weather (Keeps Rainin' All the Time)...Ethel Waters (1933)",
 '6. Begin the Beguine...Artie Shaw (1938)',
 '7. The Way You Look Tonight...Fred Astaire with Johnny Green (1936)',
 '8. Pennies from Heaven...Bing Crosby with George Stoll (1936)',
 '9. A-Tisket, A-Tasket...Ella Fitzgerald with Chick Webb (1938)',
 '10. All the Things You Are...Tommy Dorsey with Jack Leonard (1939)',
 '11. Silent Night...Bing Crosby (1935)',
 '12. Smoke Gets in Your Eyes…Paul Whiteman with Bob Lawrence (1933)',
 '13. Mood Indigo...Duke Ellington (1931)',
 '14. I Got Rhythm...Red Nichols (1930)',
 '15. Deep Purple...Larry Clinton with Bea Wain (1939)',
 '16. Happy Days Are Here Again...Ben Selvin (1930)',
 '17. God Bless America...Kate Smith (1939)',
 '18. Brother, Can You Spare a Dime?...Rud

In [202]:
def list_songs_url(url):
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    tags = soup.find_all(name="p")
    
    #Separa en elementos de una lista las canciones.
    songs = tags[5].text.split('\n')
    
    #Elimina los espacios de los extremos de cada elemento de la lista si
    #los hubiese y devuelve una lista nueva con cada elemento
    songs = [i.strip() for i in songs]
    
    #Elimina los elementos vacios de la lista
    songs = list(filter(None, songs))
    return songs

list_songs_url(url_dave)

['1. Over the Rainbow...Judy Garland (1939)',
 '2. Night and Day...Fred Astaire with Leo Reisman (1932)',
 '3. In the Mood...Glenn Miller (1939)',
 '4. Cheek to Cheek...Fred Astaire with Leo Reisman (1935)',
 "5. Stormy Weather (Keeps Rainin' All the Time)...Ethel Waters (1933)",
 '6. Begin the Beguine...Artie Shaw (1938)',
 '7. The Way You Look Tonight...Fred Astaire with Johnny Green (1936)',
 '8. Pennies from Heaven...Bing Crosby with George Stoll (1936)',
 '9. A-Tisket, A-Tasket...Ella Fitzgerald with Chick Webb (1938)',
 '10. All the Things You Are...Tommy Dorsey with Jack Leonard (1939)',
 '11. Silent Night...Bing Crosby (1935)',
 '12. Smoke Gets in Your Eyes…Paul Whiteman with Bob Lawrence (1933)',
 '13. Mood Indigo...Duke Ellington (1931)',
 '14. I Got Rhythm...Red Nichols (1930)',
 '15. Deep Purple...Larry Clinton with Bea Wain (1939)',
 '16. Happy Days Are Here Again...Ben Selvin (1930)',
 '17. God Bless America...Kate Smith (1939)',
 '18. Brother, Can You Spare a Dime?...Rud

In [196]:
test_str = '81. The Peanut Vendor (El Manisero)...Don Azpiazu with Antonio Machin (1930)'

#Recibe un string y devuelve el a;o de el string de la cancion
def extract_year(song_string):
    return song_string[-5:-1]

extract_year(test_str)

'1930'

In [168]:
#Recibe un string y devuelve el orden de la cancion
def extract_order(song_string):
    return song_string[:3].replace('.','').strip()

extract_order(test_str)

'81'

In [192]:
#Recibe un string y devuelve el titulo de la cancion
def extract_title(song_string):
    return re.findall('\d+.\s(.*)(?:\.\.\.|\…)',song_string)[0]

extract_title(test_str)

'The Peanut Vendor (El Manisero)'

In [195]:
#Recibe un string y devuelve el autor de la cancion
def extract_author(song_string):
    return re.findall('(?:\.\.\.|\…)(.*)\(',song_string)[0]

extract_author(test_str)

'Don Azpiazu with Antonio Machin '

In [203]:
def ext_data_songs(url_dave):
    list_songs_dave = list_songs_url(url_dave)
    formated_songs = []
    for i in list_songs_dave:
        year = extract_year(i)
        title = extract_title(i)
        author = extract_author(i)
        order = extract_order(i)
        
        song = {
            'year':year,
            'title':title,
            'author':author,
            'order':order,
        }
        formated_songs.append(song)
    return formated_songs

ext_data_songs(url_dave)

[{'year': '1939',
  'title': 'Over the Rainbow',
  'author': 'Judy Garland ',
  'order': '1'},
 {'year': '1932',
  'title': 'Night and Day',
  'author': 'Fred Astaire with Leo Reisman ',
  'order': '2'},
 {'year': '1939',
  'title': 'In the Mood',
  'author': 'Glenn Miller ',
  'order': '3'},
 {'year': '1935',
  'title': 'Cheek to Cheek',
  'author': 'Fred Astaire with Leo Reisman ',
  'order': '4'},
 {'year': '1933',
  'title': "Stormy Weather (Keeps Rainin' All the Time)",
  'author': 'Ethel Waters ',
  'order': '5'},
 {'year': '1938',
  'title': 'Begin the Beguine',
  'author': 'Artie Shaw ',
  'order': '6'},
 {'year': '1936',
  'title': 'The Way You Look Tonight',
  'author': 'Fred Astaire with Johnny Green ',
  'order': '7'},
 {'year': '1936',
  'title': 'Pennies from Heaven',
  'author': 'Bing Crosby with George Stoll ',
  'order': '8'},
 {'year': '1938',
  'title': 'A-Tisket, A-Tasket',
  'author': 'Ella Fitzgerald with Chick Webb ',
  'order': '9'},
 {'year': '1939',
  'title':

In [204]:
url_dave_1 = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1940-1949.html'

ext_data_songs(url_dave_1)

IndexError: list index out of range